**Import dependencies**

In [7]:
import pandas as pd
import re
import numpy as np
import sklearn
import imblearn

**Read in manga dataframe from CSV**

In [8]:
df_original = pd.read_csv('manga.csv', na_values = ['','nan','None'])

**Data wrangling** 

Replace NaNs with zeroes

In [9]:
numeric_columns = df_original.select_dtypes(include=['number']).columns

updated_df_1 = df_original
updated_df_1[numeric_columns] = updated_df_1[numeric_columns].fillna(value=0)

Adding single column indicating which demographic the manga belongs to and deleting separate columns specifying this

In [10]:
demographic_labels = ["Shounen","Shoujo","Seinen","Josei"]
demo_only_df = updated_df_1.loc[:,demographic_labels]

# Finding instances with no clear demographic tag
total_pct_per_row = demo_only_df.sum(axis=1).tolist()
indices = [index for index, element in enumerate(total_pct_per_row) if element == 0]

# Finding best fit for each row, aside from no-tag cases
demo_column_1 = demo_only_df.idxmax(axis=1).tolist()
demo_column_2 = demo_column_1

# Replacing best fit demographic with "Unknown" in cases with no clear demographic tag
for index in indices:
    demo_column_2[index] = "Unknown"

# Adding column
updated_df_2 = updated_df_1
updated_df_2["Demographic"] = demo_column_2

# Removing demographic tag columns
updated_df_3 = updated_df_2.drop(demographic_labels, axis=1)

/tmp/ipykernel_207407/1673362650.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  updated_df_2["Demographic"] = demo_column_2


Generating popularity column by summing all status columns, recalculating status raw numbers to percentage of overall statuses

In [11]:
# Getting current list of columns
columns_list_v3 = updated_df_3.columns.tolist()

# Creating list of status columns only
status_regex = re.compile("status_")
status_raws_cols = list(filter(status_regex.match, columns_list_v3))

# Summing status columns only
popularity_list = updated_df_3[status_raws_cols].sum(axis=1).tolist()

# Adding new column
updated_df_4 = updated_df_3
updated_df_4["Popularity"] = popularity_list

# Recalculating statuses to be a percentage of popularity
for column in status_raws_cols:
    updated_df_4 = updated_df_4.assign(**{column: updated_df_4[column]/updated_df_4["Popularity"]})

# Renaming status columns to reflect their new meaning (this also renames score columns, which will be updated later)
updated_df_4.columns = updated_df_4.columns.str.replace("_count", "_pct", regex=False)

Calculating total number of ratings across each manga, converting the count from each score bin into a percentage of the total ratings, and then transforming the total number of ratings into a percentage of the total popularity of the manga.

In [12]:
# Getting current list of columns
columns_list_v4 = updated_df_4.columns.tolist()

# Creating list of score columns only
score_regex = re.compile("scored_")
score_raws_cols = list(filter(score_regex.match, columns_list_v4))

# Summing score columns only
score_list = updated_df_4[score_raws_cols].sum(axis=1).tolist()

# Adding new column
updated_df_5 = updated_df_4
updated_df_5["Scored_Percentage"] = score_list

# Recalculating statuses to be a percentage of popularity
for column in score_raws_cols:
    updated_df_5 = updated_df_5.assign(**{column: updated_df_5[column]/updated_df_5["Scored_Percentage"]})

updated_df_5 = updated_df_5.assign(Scored_Percentage = updated_df_5["Scored_Percentage"]/updated_df_5["Popularity"])

Transforming favorites column into a percentage of popularity as well

In [13]:
updated_df_6 = updated_df_5.assign(favorites = updated_df_5["favorites"]/updated_df_5["Popularity"])

Calculating run length for completed works, filling -1 for ongoing works

In [16]:
updated_df_7 = updated_df_6.assign(run_length = updated_df_6["end_date_days"]-updated_df_6["start_date_days"])
updated_df_7["run_length"] = updated_df_7["run_length"].where(cond=updated_df_7["run_length"]>-1, other=-1)

**Building First Classifier**

Splitting into training and testing sets

In [25]:
training_df = updated_df_7.query("Demographic != 'Unknown'")
training_x = training_df.drop(columns=["Demographic"])
training_y = training_df[["Demographic"]]
testing_df = updated_df_7.query("Demographic == 'Unknown'")

Creating balanced random forest model, checking results (note: not yet implemented because columns need to be removed and/or one-hot encoded)

In [28]:
#rfBaseline = imblearn.ensemble.BalancedRandomForestClassifier(oob_score = True)

#rfBaseline.fit(training_x, training_y)

#rfBaseline.score()

/home/zach/anaconda3/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/zach/anaconda3/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/zach/anaconda3/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  wa

ValueError: could not convert string to float: "A Centaur's Life"